ways of transforming: using data frames /data sets or rdds(quicker)

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
%fs ls dbfs:///FileStore/tables


path,name,size,modificationTime
dbfs:/FileStore/tables/Address-1.xlsx,Address-1.xlsx,151315,1706076528000
dbfs:/FileStore/tables/Address-2.xlsx,Address-2.xlsx,151315,1706078350000
dbfs:/FileStore/tables/Address-3.xlsx,Address-3.xlsx,151315,1706078995000
dbfs:/FileStore/tables/Address-4.xlsx,Address-4.xlsx,151315,1706117061000
dbfs:/FileStore/tables/Address.xlsx,Address.xlsx,151315,1706073398000
dbfs:/FileStore/tables/Detail.csv,Detail.csv,208476,1706073642000
dbfs:/FileStore/tables/Drugs_package.csv,Drugs_package.csv,27234691,1707889055000
dbfs:/FileStore/tables/Drugs_product.csv,Drugs_product.csv,38743242,1707889061000
dbfs:/FileStore/tables/contactinfo-1.txt,contactinfo-1.txt,49969,1706078039000
dbfs:/FileStore/tables/contactinfo.txt,contactinfo.txt,49969,1706077759000


In [0]:
df_csv = spark.read.format("csv").load("dbfs:///FileStore/tables/csv/batch.csv")

In [0]:
type(df_csv)

Out[4]: pyspark.sql.dataframe.DataFrame

In [0]:
df_csv.show()

+---+-----+----------+----+------+----------+
|_c0|  _c1|       _c2| _c3|   _c4|       _c5|
+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+



for displaying header

In [0]:
df_csv = spark.read.format("csv").option("header", True).load("dbfs:///FileStore/tables/csv/batch.csv")

In [0]:
df_csv.show()

+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
+---+-----+----------+----+------+----------+
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+



order of the data is acc to rows in CSV file

In [0]:
df_csv.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- age: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- department: string (nullable = true)



for specifying data type 

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
schema = StructType(
    [
        StructField("id", IntegerType()),
        StructField("name", StringType()),
        StructField("dob", DateType()),
        StructField("age", IntegerType()),
        StructField("salary", IntegerType()),
        StructField("department", StringType()),
    ]
)

In [0]:
df_csv = spark.read.format("csv").schema(schema).option("header", True).load("dbfs:///FileStore/tables/csv/batch.csv")

In [0]:
df_csv.printSchema()
df_csv.show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)

+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
+---+-----+----------+----+------+----------+
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+



In [0]:
df_csv = spark.read.format("csv").schema(schema).option("header", True).option("inferSchema", True).load("dbfs:///FileStore/tables/csv/batch.csv")

In [0]:
df_csv.show()

+---+-----+----------+----+------+----------+
| id| name|       dob| age|salary|department|
+---+-----+----------+----+------+----------+
|  1| John|1992-05-12|  30| 70000|        IT|
|  2|Alice|1997-02-28|  25| 60000|        HR|
|  3|  Bob|      null|null| 80000|        IT|
|  4|Emily|1994-11-22|  28| 65000|   Finance|
+---+-----+----------+----+------+----------+



Extact json

In [0]:
df_json = spark.read.format("json").schema(schema).load("dbfs:///FileStore/tables/json").orderBy("id") #sets header auto no order of data, 
df_json.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)



In [0]:
df_json.show()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
|  9| James|1983-10-14|  39| 87000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
+---+------+----------+----+------+----------+



In [0]:
df_json.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)



In [0]:
df = df_csv.union(df_json)
df.show()

# issue in data

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
|  9| James|1983-10-14|  39| 87000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
+---+------+----------+----+------+----------+



In [0]:
print(df.count())

17


In [0]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)



In [0]:
print(df_csv.columns, df_json.columns) #to prints columns list

['id', 'name', 'dob', 'age', 'salary', 'department'] ['id', 'name', 'dob', 'age', 'salary', 'department']


In [0]:
# for seting columns order of df_json same as df_csv 
df_json = df_json.select(df_csv.columns)
df_json.show()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
|  9| James|1983-10-14|  39| 87000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
+---+------+----------+----+------+----------+



In [0]:
print(df_csv.columns, df_json.columns) #to prints columns list

['id', 'name', 'dob', 'age', 'salary', 'department'] ['id', 'name', 'dob', 'age', 'salary', 'department']


In [0]:
df = df_csv.union(df_json)
df.show()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
|  9| James|1983-10-14|  39| 87000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
+---+------+----------+----+------+----------+



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_parquet = spark.read.format("parquet").load('dbfs:///FileStore/tables/parquet')

# spark.read.parquet('dbfs:///FileStore/tables/parquet')

In [0]:
df_parquet.show()

+----------+-------+--------+
|department|manager|    lead|
+----------+-------+--------+
|   Finance|  Megan|   Molly|
|        HR|   Brad|   Brian|
|        IT|  Chris|Chandler|
|  Delivery|   Leon|  Louise|
+----------+-------+--------+



in pyspark UnionAll() is depricated . UnionAll() and Union() both act as UnionAll(), duplicates are not truncated. 

In [0]:
df = df.dropDuplicates()
print(df.count())

12


In [0]:
df.show()

+---+------+----------+----+------+----------+
| id|  name|       dob| age|salary|department|
+---+------+----------+----+------+----------+
|  3|   Bob|      null|null| 80000|        IT|
|  4| Emily|1994-11-22|  28| 65000|   Finance|
|  2| Alice|1997-02-28|  25| 60000|        HR|
|  1|  John|1992-05-12|  30| 70000|        IT|
|  5| David|1981-12-18|  41| 90000|        HR|
|  6| Susan|1989-07-05|  33| 75000|   Finance|
|  7|  Mike|1976-03-15|  46| 95000|        IT|
| 10|Sophie|1992-06-30|  30| 62000|   Finance|
|  2| Alice|1997-02-28|  25| 90000|   Finance|
|  4| Emily|1994-11-22|  28| 70000|   Finance|
|  9| James|1983-10-14|  39| 87000|        IT|
|  8|  Lisa|1995-08-20|  27| 58000|        HR|
+---+------+----------+----+------+----------+



Trasformation

SELECT

In [0]:
from pyspark.sql import functions as F #F.col(), F.max(), F.min()
# from pyspark.sql.functions import *
# from pyspark.sql.functions import col, max, min


In [0]:
df.select(
    "salary",
    "age"
).show()
df.select(
    ["salary", "age"]
).show()

df.select(
    df.salary,
    df.age
).show()

df.select(
    F.col("salary"),
    F.col("age")
)

+------+----+
|salary| age|
+------+----+
| 80000|null|
| 65000|  28|
| 60000|  25|
| 70000|  30|
| 90000|  41|
| 75000|  33|
| 95000|  46|
| 62000|  30|
| 90000|  25|
| 70000|  28|
| 87000|  39|
| 58000|  27|
+------+----+

+------+----+
|salary| age|
+------+----+
| 80000|null|
| 65000|  28|
| 60000|  25|
| 70000|  30|
| 90000|  41|
| 75000|  33|
| 95000|  46|
| 62000|  30|
| 90000|  25|
| 70000|  28|
| 87000|  39|
| 58000|  27|
+------+----+

+------+----+
|salary| age|
+------+----+
| 80000|null|
| 65000|  28|
| 60000|  25|
| 70000|  30|
| 90000|  41|
| 75000|  33|
| 95000|  46|
| 62000|  30|
| 90000|  25|
| 70000|  28|
| 87000|  39|
| 58000|  27|
+------+----+

Out[30]: DataFrame[salary: int, age: int]

In [0]:

df.select(
    df.salary +.05 *df.salary,
    F.year(F.current_timestamp())- F.year(F.col("dob")),
    F.year(F.current_timestamp())- F.year("dob")
).show()

+--------------------------+---------------------------------------+---------------------------------------+
|(salary + (salary * 0.05))|(year(current_timestamp()) - year(dob))|(year(current_timestamp()) - year(dob))|
+--------------------------+---------------------------------------+---------------------------------------+
|                   84000.0|                                   null|                                   null|
|                   68250.0|                                     30|                                     30|
|                   63000.0|                                     27|                                     27|
|                   73500.0|                                     32|                                     32|
|                   94500.0|                                     43|                                     43|
|                   78750.0|                                     35|                                     35|
|                  

In [0]:
print(F.year(F.current_timestamp()))

Column<'year(current_timestamp())'>


Alias, select, expr

In [0]:
df.select(
    # "*",
    F.col("*"),
    df.salary +.05 *df.salary,
    (df.salary +.05 *df.salary).alias("salary_raise"),
    F.expr("salary +.05 * salary").alias("salary"),
    (
        F.year(F.current_timestamp())- F.year(F.col("dob"))
    ).alias("age"),
).show()

+---+------+----------+----+------+----------+--------------------------+------------+--------+----+
| id|  name|       dob| age|salary|department|(salary + (salary * 0.05))|salary_raise|  salary| age|
+---+------+----------+----+------+----------+--------------------------+------------+--------+----+
|  3|   Bob|      null|null| 80000|        IT|                   84000.0|     84000.0|84000.00|null|
|  4| Emily|1994-11-22|  28| 65000|   Finance|                   68250.0|     68250.0|68250.00|  30|
|  2| Alice|1997-02-28|  25| 60000|        HR|                   63000.0|     63000.0|63000.00|  27|
|  1|  John|1992-05-12|  30| 70000|        IT|                   73500.0|     73500.0|73500.00|  32|
|  5| David|1981-12-18|  41| 90000|        HR|                   94500.0|     94500.0|94500.00|  43|
|  6| Susan|1989-07-05|  33| 75000|   Finance|                   78750.0|     78750.0|78750.00|  35|
|  7|  Mike|1976-03-15|  46| 95000|        IT|                   99750.0|     99750.0|99750

In [0]:
df.select(
    df.salary +.05 *df.salary,
    (df.salary +.05 *df.salary).alias("salary_raise"),
    F.expr("salary +.05 * salary").alias("salary"),
    (
        F.year(F.current_timestamp())- F.year(F.col("dob"))
    ).alias("age"),
).show()

+--------------------------+------------+--------+----+
|(salary + (salary * 0.05))|salary_raise|  salary| age|
+--------------------------+------------+--------+----+
|                   84000.0|     84000.0|84000.00|null|
|                   68250.0|     68250.0|68250.00|  30|
|                   63000.0|     63000.0|63000.00|  27|
|                   73500.0|     73500.0|73500.00|  32|
|                   94500.0|     94500.0|94500.00|  43|
|                   78750.0|     78750.0|78750.00|  35|
|                   99750.0|     99750.0|99750.00|  48|
|                   65100.0|     65100.0|65100.00|  32|
|                   94500.0|     94500.0|94500.00|  27|
|                   73500.0|     73500.0|73500.00|  30|
|                   91350.0|     91350.0|91350.00|  41|
|                   60900.0|     60900.0|60900.00|  29|
+--------------------------+------------+--------+----+



withcolumns: appends column

In [0]:
#either

# df=df.withColumn(
#     "salary_raise",
#     (F.col("salary") +0.5 *F.col("salary"))
# )
# df=df.withColumn(
#     "salary_raise_10_pers",
#     (F.col("salary") +.1 *F.col("salary"))
# )

#or
df = df.withColumn(
    "salary_raise",
    (F.col("salary") +0.5 *F.col("salary"))
).withColumn(
    "salary_raise_10_pers",
    (F.col("salary") +.1 *F.col("salary"))
).withColumn(
    "salary_raise_7_pers",
    (F.col("salary") +.7 *F.col("salary"))
)
df.show()

+---+------+----------+----+------+----------+------------+--------------------+-------------------+
| id|  name|       dob| age|salary|department|salary_raise|salary_raise_10_pers|salary_raise_7_pers|
+---+------+----------+----+------+----------+------------+--------------------+-------------------+
|  3|   Bob|      null|null| 80000|        IT|    120000.0|             88000.0|           136000.0|
|  4| Emily|1994-11-22|  28| 65000|   Finance|     97500.0|             71500.0|           110500.0|
|  2| Alice|1997-02-28|  25| 60000|        HR|     90000.0|             66000.0|           102000.0|
|  1|  John|1992-05-12|  30| 70000|        IT|    105000.0|             77000.0|           119000.0|
|  5| David|1981-12-18|  41| 90000|        HR|    135000.0|             99000.0|           153000.0|
|  6| Susan|1989-07-05|  33| 75000|   Finance|    112500.0|             82500.0|           127500.0|
|  7|  Mike|1976-03-15|  46| 95000|        IT|    142500.0|            104500.0|           

with columns used for reaplcing existing columns and append multiple columns

In [0]:
df = df.withColumns(
    {
        "salary_raise": F.col("salary") + .05*F.col("salary"),
        "age":   F.year(F.current_timestamp())- F.year(F.col("dob"))
    }
)

In [0]:
df.show()

+---+------+----------+----+------+----------+------------+--------------------+-------------------+
| id|  name|       dob| age|salary|department|salary_raise|salary_raise_10_pers|salary_raise_7_pers|
+---+------+----------+----+------+----------+------------+--------------------+-------------------+
|  3|   Bob|      null|null| 80000|        IT|     84000.0|             88000.0|           136000.0|
|  4| Emily|1994-11-22|  30| 65000|   Finance|     68250.0|             71500.0|           110500.0|
|  2| Alice|1997-02-28|  27| 60000|        HR|     63000.0|             66000.0|           102000.0|
|  1|  John|1992-05-12|  32| 70000|        IT|     73500.0|             77000.0|           119000.0|
|  5| David|1981-12-18|  43| 90000|        HR|     94500.0|             99000.0|           153000.0|
|  6| Susan|1989-07-05|  35| 75000|   Finance|     78750.0|             82500.0|           127500.0|
|  7|  Mike|1976-03-15|  48| 95000|        IT|     99750.0|            104500.0|           

Filter : equivalnet to where

In [0]:
df.filter(
    F.col("salary_raise")>=75000
).show()

+---+-----+----------+----+------+----------+------------+--------------------+-------------------+
| id| name|       dob| age|salary|department|salary_raise|salary_raise_10_pers|salary_raise_7_pers|
+---+-----+----------+----+------+----------+------------+--------------------+-------------------+
|  3|  Bob|      null|null| 80000|        IT|     84000.0|             88000.0|           136000.0|
|  5|David|1981-12-18|  43| 90000|        HR|     94500.0|             99000.0|           153000.0|
|  6|Susan|1989-07-05|  35| 75000|   Finance|     78750.0|             82500.0|           127500.0|
|  7| Mike|1976-03-15|  48| 95000|        IT|     99750.0|            104500.0|           161500.0|
|  2|Alice|1997-02-28|  27| 90000|   Finance|     94500.0|             99000.0|           153000.0|
|  9|James|1983-10-14|  41| 87000|        IT|     91350.0|             95700.0|           147900.0|
+---+-----+----------+----+------+----------+------------+--------------------+-------------------+


In [0]:
df.where(
    F.col("salary_raise")>=75000
).show()

+---+-----+----------+----+------+----------+------------+--------------------+-------------------+
| id| name|       dob| age|salary|department|salary_raise|salary_raise_10_pers|salary_raise_7_pers|
+---+-----+----------+----+------+----------+------------+--------------------+-------------------+
|  3|  Bob|      null|null| 80000|        IT|     84000.0|             88000.0|           136000.0|
|  5|David|1981-12-18|  43| 90000|        HR|     94500.0|             99000.0|           153000.0|
|  6|Susan|1989-07-05|  35| 75000|   Finance|     78750.0|             82500.0|           127500.0|
|  7| Mike|1976-03-15|  48| 95000|        IT|     99750.0|            104500.0|           161500.0|
|  2|Alice|1997-02-28|  27| 90000|   Finance|     94500.0|             99000.0|           153000.0|
|  9|James|1983-10-14|  41| 87000|        IT|     91350.0|             95700.0|           147900.0|
+---+-----+----------+----+------+----------+------------+--------------------+-------------------+


wehn otherwsie


In [0]:
df = df.withColumn(
    "age_group",
    F.when(
        F.col("age") <=20,
        "Upto 20"
    ).when(
        (
            (F.col("age") >20) &
            (F.col("age") <=30)
        ),
        "21 to 30"
    ).when (
        (
            (F.col("age") > 30) &
            (F.col("age") <= 40)
        ),
        "31 to 40"
    ).otherwise(
        "More than 40"
    )
)

In [0]:
df = df.withColumn(
    "age_group",
    F.when(
        F.col("age") <=20,
        "Upto 20"
    ).when(
        (
            (F.col("age") >20) &
            (F.col("age") <=30)
        ),
        "21 to 30"
    ).when (
        (
            (F.col("age") > 30) &
            (F.col("age") <= 40)
        ),
        F.col("name") #col value can be assign instead of contant value
    ).otherwise(
        "More than 40"
    )
)

In [0]:
df.show()

+---+------+----------+----+------+----------+------------+--------------------+-------------------+------------+
| id|  name|       dob| age|salary|department|salary_raise|salary_raise_10_pers|salary_raise_7_pers|   age_group|
+---+------+----------+----+------+----------+------------+--------------------+-------------------+------------+
|  3|   Bob|      null|null| 80000|        IT|     84000.0|             88000.0|           136000.0|More than 40|
|  4| Emily|1994-11-22|  30| 65000|   Finance|     68250.0|             71500.0|           110500.0|    21 to 30|
|  2| Alice|1997-02-28|  27| 60000|        HR|     63000.0|             66000.0|           102000.0|    21 to 30|
|  1|  John|1992-05-12|  32| 70000|        IT|     73500.0|             77000.0|           119000.0|        John|
|  5| David|1981-12-18|  43| 90000|        HR|     94500.0|             99000.0|           153000.0|More than 40|
|  6| Susan|1989-07-05|  35| 75000|   Finance|     78750.0|             82500.0|        

In [0]:
df.withColumns(
    {
        "company": F.lit("Abacus Insights")
    }
).show()

+---+------+----------+----+------+----------+------------+--------------------+-------------------+------------+---------------+
| id|  name|       dob| age|salary|department|salary_raise|salary_raise_10_pers|salary_raise_7_pers|   age_group|        company|
+---+------+----------+----+------+----------+------------+--------------------+-------------------+------------+---------------+
|  3|   Bob|      null|null| 80000|        IT|     84000.0|             88000.0|           136000.0|More than 40|Abacus Insights|
|  4| Emily|1994-11-22|  30| 65000|   Finance|     68250.0|             71500.0|           110500.0|    21 to 30|Abacus Insights|
|  2| Alice|1997-02-28|  27| 60000|        HR|     63000.0|             66000.0|           102000.0|    21 to 30|Abacus Insights|
|  1|  John|1992-05-12|  32| 70000|        IT|     73500.0|             77000.0|           119000.0|        John|Abacus Insights|
|  5| David|1981-12-18|  43| 90000|        HR|     94500.0|             99000.0|          

nulls

In [0]:
df = df.withColumns(
    {
        "age":F.coalesce(
            F.year(F.current_timestamp()) - F.year(F.col("dob")),
            F.lit(0),
        ),
        "has_dob_1": ~(F.isnull("dob")),
        "has_dob_2":F.col("dob").isNotNull(),
    }
)

# ~ => !=

In [0]:
df.show()

+---+------+----------+---+------+----------+------------+--------------------+-------------------+------------+---------+---------+
| id|  name|       dob|age|salary|department|salary_raise|salary_raise_10_pers|salary_raise_7_pers|   age_group|has_dob_1|has_dob_2|
+---+------+----------+---+------+----------+------------+--------------------+-------------------+------------+---------+---------+
|  3|   Bob|      null|  0| 80000|        IT|     84000.0|             88000.0|           136000.0|More than 40|    false|    false|
|  4| Emily|1994-11-22| 30| 65000|   Finance|     68250.0|             71500.0|           110500.0|    21 to 30|     true|     true|
|  2| Alice|1997-02-28| 27| 60000|        HR|     63000.0|             66000.0|           102000.0|    21 to 30|     true|     true|
|  1|  John|1992-05-12| 32| 70000|        IT|     73500.0|             77000.0|           119000.0|        John|     true|     true|
|  5| David|1981-12-18| 43| 90000|        HR|     94500.0|           

drop columns

In [0]:
df =df.drop("salary_raise_7_pers", "salary_raise_10_pers")

In [0]:
df.show()

+---+------+----------+---+------+----------+------------+------------+---------+---------+
| id|  name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|
+---+------+----------+---+------+----------+------------+------------+---------+---------+
|  3|   Bob|      null|  0| 80000|        IT|     84000.0|More than 40|    false|    false|
|  4| Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|
|  2| Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|
|  1|  John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true|
|  5| David|1981-12-18| 43| 90000|        HR|     94500.0|More than 40|     true|     true|
|  6| Susan|1989-07-05| 35| 75000|   Finance|     78750.0|       Susan|     true|     true|
|  7|  Mike|1976-03-15| 48| 95000|        IT|     99750.0|More than 40|     true|     true|
| 10|Sophie|1992-06-30| 32| 62000|   Finance|     65100.0|      Sophie|     true

In [0]:
df=df.withColumnRenamed("name", "first_name")

In [0]:
df.show()

+---+----------+----------+---+------+----------+------------+------------+---------+---------+
| id|first_name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|
+---+----------+----------+---+------+----------+------------+------------+---------+---------+
|  3|       Bob|      null|  0| 80000|        IT|     84000.0|More than 40|    false|    false|
|  4|     Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|
|  2|     Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|
|  1|      John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true|
|  5|     David|1981-12-18| 43| 90000|        HR|     94500.0|More than 40|     true|     true|
|  6|     Susan|1989-07-05| 35| 75000|   Finance|     78750.0|       Susan|     true|     true|
|  7|      Mike|1976-03-15| 48| 95000|        IT|     99750.0|More than 40|     true|     true|
| 10|    Sophie|1992-06-30| 32| 62000|  

partition by 

In [0]:
from pyspark.sql import Window

In [0]:
window =Window.partitionBy("age_group")
df.withColumn(
    "min_age_in_group",
    F.min("age").over(window)
).show()

+---+----------+----------+---+------+----------+------------+------------+---------+---------+----------------+
| id|first_name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|min_age_in_group|
+---+----------+----------+---+------+----------+------------+------------+---------+---------+----------------+
|  4|     Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|              27|
|  2|     Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|              27|
|  2|     Alice|1997-02-28| 27| 90000|   Finance|     94500.0|    21 to 30|     true|     true|              27|
|  4|     Emily|1994-11-22| 30| 70000|   Finance|     73500.0|    21 to 30|     true|     true|              27|
|  8|      Lisa|1995-08-20| 29| 58000|        HR|     60900.0|    21 to 30|     true|     true|              27|
|  1|      John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true| 

In [0]:
df.show()

+---+----------+----------+---+------+----------+------------+------------+---------+---------+
| id|first_name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|
+---+----------+----------+---+------+----------+------------+------------+---------+---------+
|  3|       Bob|      null|  0| 80000|        IT|     84000.0|More than 40|    false|    false|
|  4|     Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|
|  2|     Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|
|  1|      John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true|
|  5|     David|1981-12-18| 43| 90000|        HR|     94500.0|More than 40|     true|     true|
|  6|     Susan|1989-07-05| 35| 75000|   Finance|     78750.0|       Susan|     true|     true|
|  7|      Mike|1976-03-15| 48| 95000|        IT|     99750.0|More than 40|     true|     true|
| 10|    Sophie|1992-06-30| 32| 62000|  

sql lower_case, tables, column name, schemas, datbase names



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
ref_def = spark.read.format("parquet").load('dbfs:///FileStore/tables/parquet')

In [0]:
ref_def.show()

+----------+-------+--------+
|department|manager|    lead|
+----------+-------+--------+
|   Finance|  Megan|   Molly|
|        HR|   Brad|   Brian|
|        IT|  Chris|Chandler|
|  Delivery|   Leon|  Louise|
+----------+-------+--------+



Join

In [0]:
df.join(
    ref_def,
    "department",
    "left",
).show()

+----------+---+----------+----------+---+------+------------+------------+---------+---------+-------+--------+
|department| id|first_name|       dob|age|salary|salary_raise|   age_group|has_dob_1|has_dob_2|manager|    lead|
+----------+---+----------+----------+---+------+------------+------------+---------+---------+-------+--------+
|        IT|  3|       Bob|      null|  0| 80000|     84000.0|More than 40|    false|    false|  Chris|Chandler|
|   Finance|  4|     Emily|1994-11-22| 30| 65000|     68250.0|    21 to 30|     true|     true|  Megan|   Molly|
|        HR|  2|     Alice|1997-02-28| 27| 60000|     63000.0|    21 to 30|     true|     true|   Brad|   Brian|
|        IT|  1|      John|1992-05-12| 32| 70000|     73500.0|        John|     true|     true|  Chris|Chandler|
|        HR|  5|     David|1981-12-18| 43| 90000|     94500.0|More than 40|     true|     true|   Brad|   Brian|
|   Finance|  6|     Susan|1989-07-05| 35| 75000|     78750.0|       Susan|     true|     true| 

In [0]:
df.join(
    ref_def,
    df.department == ref_def.department,
    "left",
).show()

+---+----------+----------+---+------+----------+------------+------------+---------+---------+----------+-------+--------+
| id|first_name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|department|manager|    lead|
+---+----------+----------+---+------+----------+------------+------------+---------+---------+----------+-------+--------+
|  3|       Bob|      null|  0| 80000|        IT|     84000.0|More than 40|    false|    false|        IT|  Chris|Chandler|
|  4|     Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|   Finance|  Megan|   Molly|
|  2|     Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|        HR|   Brad|   Brian|
|  1|      John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true|        IT|  Chris|Chandler|
|  5|     David|1981-12-18| 43| 90000|        HR|     94500.0|More than 40|     true|     true|        HR|   Brad|   Brian|
|  6|   

In [0]:
df = df.alias("main")
ref_def =ref_def.alias("references")

In [0]:
df.join(
    ref_def,
    F.col("main.department") == F.col("references.department"),
    "left",
).show()

+---+----------+----------+---+------+----------+------------+------------+---------+---------+----------+-------+--------+
| id|first_name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|department|manager|    lead|
+---+----------+----------+---+------+----------+------------+------------+---------+---------+----------+-------+--------+
|  3|       Bob|      null|  0| 80000|        IT|     84000.0|More than 40|    false|    false|        IT|  Chris|Chandler|
|  4|     Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|   Finance|  Megan|   Molly|
|  2|     Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|        HR|   Brad|   Brian|
|  1|      John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true|        IT|  Chris|Chandler|
|  5|     David|1981-12-18| 43| 90000|        HR|     94500.0|More than 40|     true|     true|        HR|   Brad|   Brian|
|  6|   

In [0]:
df.join(
    ref_def.select(
        "manager",
        "lead",
        F.col("department").alias("reference_department")
    ).alias("reference"),
    F.col("main.department") == F.col("reference.reference_department"),
    "left",
).show()

+---+----------+----------+---+------+----------+------------+------------+---------+---------+-------+--------+--------------------+
| id|first_name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|manager|    lead|reference_department|
+---+----------+----------+---+------+----------+------------+------------+---------+---------+-------+--------+--------------------+
|  3|       Bob|      null|  0| 80000|        IT|     84000.0|More than 40|    false|    false|  Chris|Chandler|                  IT|
|  4|     Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|  Megan|   Molly|             Finance|
|  2|     Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|   Brad|   Brian|                  HR|
|  1|      John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true|  Chris|Chandler|                  IT|
|  5|     David|1981-12-18| 43| 90000|        HR|     94500.0|

In [0]:
df.join(
    ref_def.select(
        "manager",
        "lead",
        F.col("department").alias("reference_department")
    ).alias("reference"),
    F.col("main.department") == F.col("reference.reference_department"),
    "left",
).show()

+---+----------+----------+---+------+----------+------------+------------+---------+---------+-------+--------+--------------------+
| id|first_name|       dob|age|salary|department|salary_raise|   age_group|has_dob_1|has_dob_2|manager|    lead|reference_department|
+---+----------+----------+---+------+----------+------------+------------+---------+---------+-------+--------+--------------------+
|  3|       Bob|      null|  0| 80000|        IT|     84000.0|More than 40|    false|    false|  Chris|Chandler|                  IT|
|  4|     Emily|1994-11-22| 30| 65000|   Finance|     68250.0|    21 to 30|     true|     true|  Megan|   Molly|             Finance|
|  2|     Alice|1997-02-28| 27| 60000|        HR|     63000.0|    21 to 30|     true|     true|   Brad|   Brian|                  HR|
|  1|      John|1992-05-12| 32| 70000|        IT|     73500.0|        John|     true|     true|  Chris|Chandler|                  IT|
|  5|     David|1981-12-18| 43| 90000|        HR|     94500.0|

Load(Save) Data

In [0]:
df.write.mode("overwrite").format("csv").save("dbfs:///FileStore/tables/final")

In [0]:
%fs ls dbfs:///FileStore/tables/final

path,name,size,modificationTime
dbfs:/FileStore/tables/final/_SUCCESS,_SUCCESS,0,1708269700000
dbfs:/FileStore/tables/final/_committed_3623868331512841782,_committed_3623868331512841782,201,1708269259000
dbfs:/FileStore/tables/final/_committed_4247167990502818000,_committed_4247167990502818000,201,1708269700000
dbfs:/FileStore/tables/final/_committed_6046920417504358138,_committed_6046920417504358138,301,1708268482000
dbfs:/FileStore/tables/final/_committed_7451016719198375504,_committed_7451016719198375504,202,1707804194000
dbfs:/FileStore/tables/final/_committed_vacuum4410862666706982587,_committed_vacuum4410862666706982587,96,1708268483000
dbfs:/FileStore/tables/final/_started_3623868331512841782,_started_3623868331512841782,0,1708269259000
dbfs:/FileStore/tables/final/_started_4247167990502818000,_started_4247167990502818000,0,1708269700000
dbfs:/FileStore/tables/final/_started_6046920417504358138,_started_6046920417504358138,0,1708268482000
dbfs:/FileStore/tables/final/part-00000-tid-4247167990502818000-f26a6208-a158-4ef9-964e-cf654ca545fa-321-1-c000.csv,part-00000-tid-4247167990502818000-f26a6208-a158-4ef9-964e-cf654ca545fa-321-1-c000.csv,716,1708269700000


In [0]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [0]:
%fs rm -r dbfs:/user/hive/warehouse/final

res5: Boolean = true

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("final")

In [0]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|    final|      false|
+--------+---------+-----------+



In [0]:
%fs ls dbfs:/user/hive/warehouse/final

path,name,size,modificationTime
dbfs:/user/hive/warehouse/final/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/final/part-00000-ae228bf0-db04-42e5-8826-d6b07cd7482f-c000.snappy.parquet,part-00000-ae228bf0-db04-42e5-8826-d6b07cd7482f-c000.snappy.parquet,3260,1708269716000


In [0]:
%sql
desc table default.final;

col_name,data_type,comment
id,int,null
first_name,string,null
dob,date,null
age,int,null
salary,int,null
department,string,null
salary_raise,double,null
age_group,string,null
has_dob_1,boolean,null
has_dob_2,boolean,null


In [0]:
%sql
desc table extended default.final;

col_name,data_type,comment
id,int,null
first_name,string,null
dob,date,null
age,int,null
salary,int,null
department,string,null
salary_raise,double,null
age_group,string,null
has_dob_1,boolean,null
has_dob_2,boolean,null


In [0]:
%sql
desc detail default.final;

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,53506fb6-f9c3-4bd1-9945-dcc34ba7c4cc,spark_catalog.default.final,null,dbfs:/user/hive/warehouse/final,2024-02-18T15:21:52.952+0000,2024-02-18T15:21:59.000+0000,List(),1,3260,Map(),1,2,"List(appendOnly, invariants)",Map()


In [0]:
%sql
SELECT * from default.final

id,first_name,dob,age,salary,department,salary_raise,age_group,has_dob_1,has_dob_2
3,Bob,null,0,80000,IT,84000.0,More than 40,false,false
4,Emily,1994-11-22,30,65000,Finance,68250.0,21 to 30,true,true
2,Alice,1997-02-28,27,60000,HR,63000.0,21 to 30,true,true
1,John,1992-05-12,32,70000,IT,73500.0,John,true,true
5,David,1981-12-18,43,90000,HR,94500.0,More than 40,true,true
6,Susan,1989-07-05,35,75000,Finance,78750.0,Susan,true,true
7,Mike,1976-03-15,48,95000,IT,99750.0,More than 40,true,true
10,Sophie,1992-06-30,32,62000,Finance,65100.0,Sophie,true,true
2,Alice,1997-02-28,27,90000,Finance,94500.0,21 to 30,true,true
4,Emily,1994-11-22,30,70000,Finance,73500.0,21 to 30,true,true


In [0]:
%sql
select * except(id, dob, age) from default.final;

first_name,salary,department,salary_raise,age_group,has_dob_1,has_dob_2
Bob,80000,IT,84000.0,More than 40,false,false
Emily,65000,Finance,68250.0,21 to 30,true,true
Alice,60000,HR,63000.0,21 to 30,true,true
John,70000,IT,73500.0,John,true,true
David,90000,HR,94500.0,More than 40,true,true
Susan,75000,Finance,78750.0,Susan,true,true
Mike,95000,IT,99750.0,More than 40,true,true
Sophie,62000,Finance,65100.0,Sophie,true,true
Alice,90000,Finance,94500.0,21 to 30,true,true
Emily,70000,Finance,73500.0,21 to 30,true,true


In [0]:
table_df = spark.table("final")

display(table_df)

id,first_name,dob,age,salary,department,salary_raise,age_group,has_dob_1,has_dob_2
3,Bob,null,0,80000,IT,84000.0,More than 40,false,false
4,Emily,1994-11-22,30,65000,Finance,68250.0,21 to 30,true,true
2,Alice,1997-02-28,27,60000,HR,63000.0,21 to 30,true,true
1,John,1992-05-12,32,70000,IT,73500.0,John,true,true
5,David,1981-12-18,43,90000,HR,94500.0,More than 40,true,true
6,Susan,1989-07-05,35,75000,Finance,78750.0,Susan,true,true
7,Mike,1976-03-15,48,95000,IT,99750.0,More than 40,true,true
10,Sophie,1992-06-30,32,62000,Finance,65100.0,Sophie,true,true
2,Alice,1997-02-28,27,90000,Finance,94500.0,21 to 30,true,true
4,Emily,1994-11-22,30,70000,Finance,73500.0,21 to 30,true,true
